In [47]:
import numpy as np
import pandas as pd

In [51]:
data = pd.read_csv("./resale-flat-prices-based-on-approval-date-2000-feb-2012.csv")
data['year'] = data.month.str.split('-').str[0]

In [52]:
low_storey = []
mid_storey = []
high_storey = []

for i in range(1, 41, 3):
    storey_range = f'{i:0>2} TO {i+2:0>2}'
    
    if i < 5:
        low_storey.append(storey_range)
    elif i < 12:
        mid_storey.append(storey_range)
    else:
        high_storey.append(storey_range)

In [53]:
data['is_low_storey'] = data.storey_range.isin(low_storey)
data['is_mid_storey'] = data.storey_range.isin(mid_storey)
data['is_high_storey'] = data.storey_range.isin(high_storey)

In [59]:
data

,month,town,flat_type,block,street_name,storey_range,floor_area_sqm,flat_model,lease_commence_date,resale_price,year,is_low_storey,is_mid_storey,is_high_storey
0,2000-01,ANG MO KIO,3 ROOM,170,ANG MO KIO AVE 4,07 TO 09,69.0,Improved,1986,147000.0,2000,False,True,False
1,2000-01,ANG MO KIO,3 ROOM,174,ANG MO KIO AVE 4,04 TO 06,61.0,Improved,1986,144000.0,2000,True,False,False
2,2000-01,ANG MO KIO,3 ROOM,216,ANG MO KIO AVE 1,07 TO 09,73.0,New Generation,1976,159000.0,2000,False,True,False
3,2000-01,ANG MO KIO,3 ROOM,215,ANG MO KIO AVE 1,07 TO 09,73.0,New Generation,1976,167000.0,2000,False,True,False
4,2000-01,ANG MO KIO,3 ROOM,218,ANG MO KIO AVE 1,07 TO 09,67.0,New Generation,1976,163000.0,2000,False,True,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
369646,2012-02,YISHUN,5 ROOM,212,YISHUN ST 21,10 TO 12,121.0,Improved,1985,476888.0,2012,False,True,False
369647,2012-02,YISHUN,5 ROOM,758,YISHUN ST 72,01 TO 03,122.0,Improved,1986,490000.0,2012,True,False,False
369648,2012-02,YISHUN,5 ROOM,873,YISHUN ST 81,01 TO 03,122.0,Improved,1988,488000.0,2012,True,False,False
369649,2012-02,YISHUN,EXECUTIVE,664,YISHUN AVE 4,07 TO 09,181.0,Apartment,1992,705000.0,2012,False,True,False


In [97]:
cols = [
    'is_low_storey',
#     'is_mid_storey',
    'is_high_storey',
    'town',
    'block',
    'flat_type',
    'resale_price'
]
d = data \
    [~data.is_mid_storey] \
    [cols]

res = d.groupby(['town', 'block', 'flat_type']).sum()

LOWER_BOUND = 5
res = res[res['is_low_storey'] >= LOWER_BOUND]
res = res[res['is_high_storey'] > LOWER_BOUND]
res

is_low_storey  is_high_storey  resale_price
town       block flat_type                                             
ANG MO KIO 130   3 ROOM                41               8    10287964.0
           134   4 ROOM                24               6     8320730.0
           178   3 ROOM                42              15    10739338.0
           218   3 ROOM                32               8     7607000.0
           259   5 ROOM                 9              21    13841888.0
...                                   ...             ...           ...
YISHUN     296   4 ROOM                37               6    12128278.0
           775   5 ROOM                20               7     8889299.0
           785   5 ROOM                15               6     6928800.0
           800   4 ROOM                22               7     7726468.0
           805   4 ROOM                14               6     4628500.0

[2173 rows x 3 columns]

In [119]:
for storey_type in ['low_storey', 'high_storey']:
    res[f'mean_{storey_type}_resale_price'] = data[data[f'is_{storey_type}']] \
        .groupby(['town', 'block', 'flat_type']) \
        .resale_price.mean()
    
res['weight'] = res[['is_low_storey', 'is_high_storey']].min(axis=1)
res['ratio_high_to_low_storey_mean_resale_price'] = res['mean_high_storey_resale_price'] / res['mean_low_storey_resale_price']
res

is_low_storey  is_high_storey  resale_price  \
town       block flat_type                                                
ANG MO KIO 130   3 ROOM                41               8    10287964.0   
           134   4 ROOM                24               6     8320730.0   
           178   3 ROOM                42              15    10739338.0   
           218   3 ROOM                32               8     7607000.0   
           259   5 ROOM                 9              21    13841888.0   
...                                   ...             ...           ...   
YISHUN     296   4 ROOM                37               6    12128278.0   
           775   5 ROOM                20               7     8889299.0   
           785   5 ROOM                15               6     6928800.0   
           800   4 ROOM                22               7     7726468.0   
           805   4 ROOM                14               6     4628500.0   

                            mean_low_storey  mean_high_storey  \
town       block flat_type                                      
ANG MO KIO 130   3 ROOM       212067.414634     199150.000000   
           134   4 ROOM       275030.416667     286666.666667   
           178   3 ROOM       186112.809524     194840.000000   
           218   3 ROOM       193640.625000     176312.500000   
           259   5 ROOM       417000.000000     480423.238095   
...                                     ...               ...   
YISHUN     296   4 ROOM       284972.378378     264050.000000   
           775   5 ROOM       321714.950000     350714.285714   
           785   5 ROOM       320653.333333     353166.666667   
           800   4 ROOM       247384.909091     326285.714286   
           805   4 ROOM       220678.571429     256500.000000   

                            mean_low_storey_resale_price  \
town       block flat_type                                 
ANG MO KIO 130   3 ROOM                    212067.414634   
           134   4 ROOM                    275030.416667   
           178   3 ROOM                    186112.809524   
           218   3 ROOM                    193640.625000   
           259   5 ROOM                    417000.000000   
...                                                  ...   
YISHUN     296   4 ROOM                    284972.378378   
           775   5 ROOM                    321714.950000   
           785   5 ROOM                    320653.333333   
           800   4 ROOM                    247384.909091   
           805   4 ROOM                    220678.571429   

                            mean_high_storey_resale_price  weight  \
town       block flat_type                                          
ANG MO KIO 130   3 ROOM                     199150.000000       8   
           134   4 ROOM                     286666.666667       6   
           178   3 ROOM                     194840.000000      15   
           218   3 ROOM                     176312.500000       8   
           259   5 ROOM                     480423.238095       9   
...                                                   ...     ...   
YISHUN     296   4 ROOM                     264050.000000       6   
           775   5 ROOM                     350714.285714       7   
           785   5 ROOM                     353166.666667       6   
           800   4 ROOM                     326285.714286       7   
           805   4 ROOM                     256500.000000       6   

                            ratio_high_to_low_storey_mean_resale_price  
town       block flat_type                                              
ANG MO KIO 130   3 ROOM                                       0.939088  
           134   4 ROOM                                       1.042309  
           178   3 ROOM                                       1.046892  
           218   3 ROOM                                       0.910514  
           259   5 ROOM                                       1.152094  
...             

In [137]:
all_types = res.groupby('town').ratio_high_to_low_storey_mean_resale_price.mean()
final = res.groupby(['town', 'flat_type']) \
    .ratio_high_to_low_storey_mean_resale_price.mean() \
    .unstack()

final['ALL'] = all_types
final \
    .to_json('processed.json')

In [107]:
r2 = res.copy()
pd.concat
r2['mean_high_storey_price'] = dfs['is_high_storey']
r2

is_low_storey  is_high_storey  resale_price  \
town       block flat_type                                                
ANG MO KIO 130   3 ROOM                41               8    10287964.0   
           134   4 ROOM                24               6     8320730.0   
           178   3 ROOM                42              15    10739338.0   
           218   3 ROOM                32               8     7607000.0   
           259   5 ROOM                 9              21    13841888.0   
...                                   ...             ...           ...   
YISHUN     296   4 ROOM                37               6    12128278.0   
           775   5 ROOM                20               7     8889299.0   
           785   5 ROOM                15               6     6928800.0   
           800   4 ROOM                22               7     7726468.0   
           805   4 ROOM                14               6     4628500.0   

                            mean_high_storey_price  
town       block flat_type                          
ANG MO KIO 130   3 ROOM              199150.000000  
           134   4 ROOM              286666.666667  
           178   3 ROOM              194840.000000  
           218   3 ROOM              176312.500000  
           259   5 ROOM              480423.238095  
...                                            ...  
YISHUN     296   4 ROOM              264050.000000  
           775   5 ROOM              350714.285714  
           785   5 ROOM              353166.666667  
           800   4 ROOM              326285.714286  
           805   4 ROOM              256500.000000  

[2173 rows x 4 columns]

In [94]:
for t in res.itertuples():
    town, block, flat_type = t.Index
    print(min(t.is_low_storey, t.is_high_storey))
    

8
6
2
2
3
15
2
4
5
2
8
1
3
5
2
9
9
7
5
3
1
3
6
6
3
1
3
1
2
10
4
3
5
3
1
13
1
10
1
13
7
3
11
4
9
2
12
12
3
9
1
4
7
5
11
13
11
9
3
15
9
10
1
10
11
1
7
6
3
2
4
8
7
1
6
4
7
7
13
13
1
18
3
8
10
5
3
12
1
4
3
4
1
8
1
7
6
1
8
8
1
4
3
12
10
6
1
11
1
8
2
9
1
4
2
5
2
10
4
3
2
3
2
4
8
9
4
1
7
7
5
10
4
1
7
3
6
6
4
4
5
4
2
8
4
9
7
11
7
13
3
5
3
17
2
13
3
2
11
1
4
7
2
4
7
5
1
3
4
9
7
9
6
2
6
1
15
10
13
1
8
3
4
8
1
2
13
4
13
9
4
10
3
9
1
11
1
10
5
9
1
15
10
3
5
10
1
11
5
1
1
1
3
1
4
2
5
3
12
1
3
3
1
4
20
4
1
3
6
11
12
13
8
4
10
8
7
1
1
2
14
1
2
16
10
9
4
13
11
10
17
10
17
1
10
17
11
25
12
19
6
27
11
2
16
26
2
9
6
7
5
9
10
8
10
15
9
6
15
11
10
11
12
11
5
1
6
18
21
18
21
9
8
8
8
25
4
11
16
3
3
3
11
1
9
27
7
14
15
4
6
2
16
6
12
1
6
14
2
4
2
26
8
24
7
8
7
3
12
38
3
8
4
8
2
2
6
6
2
12
12
1
10
1
4
6
7
4
9
4
1
5
4
3
10
3
2
2
7
6
5
8
7
1
10
4
3
5
5
12
9
3
1
4
3
1
1
6
10
9
6
8
9
5
3
2
5
3
1
18
2
2
3
9
10
3
6
10
15
25
13
7
8
8
4
11
4
5
11
3
7
6
8
6
3
10
5
21
6
6
6
6
4
5
8
4
14
3
6
7
10
13
3
6
5
4
13
3
2
8
1
10


In [44]:
low_storey

['01 TO 03', '04 TO 06']

In [45]:
mid_storey

['07 TO 09', '10 TO 12']

In [46]:
high_storey

['13 TO 15',
 '16 TO 18',
 '19 TO 21',
 '22 TO 24',
 '25 TO 27',
 '28 TO 30',
 '31 TO 33',
 '34 TO 36',
 '37 TO 39',
 '40 TO 42']